In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np

In [2]:
DATA = '/home/ike/Data/Molecular/'

train = pd.read_csv(f"{DATA}train.csv")
#train['invweight'] = train.groupby('type')['id'].transform('count')
test = pd.read_csv(f"{DATA}test.csv")
structure = pd.read_csv(f"{DATA}structures.csv")
data = pd.concat([train.drop(columns=['scalar_coupling_constant']), test], ignore_index=True)

In [3]:
data = data.merge(structure.add_suffix("_0"), how='left', left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name_0', 'atom_index_0'])\
.merge(structure.add_suffix("_1"), how='left', left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name_1', 'atom_index_1'])\
.drop(columns=['molecule_name_0', 'molecule_name_1'])

In [56]:
data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,...,dist_molecule_mean,dist_molecule_std,dist_x_molecule_min,dist_x_molecule_max,dist_y_molecule_min,dist_y_molecule_max,dist_z_molecule_min,dist_z_molecule_max,dist_molecule_min,dist_molecule_max
0,0,dsgdb9nsd_000001,1,0,1JHC,H,0.002150,-0.006031,0.001976,C,...,1.506668,0.356932,0.014849,1.552546,0.009594,1.469782,0.0017,1.783041,1.091946,1.783158
1,1,dsgdb9nsd_000001,1,2,2JHH,H,0.002150,-0.006031,0.001976,H,...,1.506668,0.356932,0.014849,1.552546,0.009594,1.469782,0.0017,1.783041,1.091946,1.783158
2,2,dsgdb9nsd_000001,1,3,2JHH,H,0.002150,-0.006031,0.001976,H,...,1.506668,0.356932,0.014849,1.552546,0.009594,1.469782,0.0017,1.783041,1.091946,1.783158
3,3,dsgdb9nsd_000001,1,4,2JHH,H,0.002150,-0.006031,0.001976,H,...,1.506668,0.356932,0.014849,1.552546,0.009594,1.469782,0.0017,1.783041,1.091946,1.783158
4,4,dsgdb9nsd_000001,2,0,1JHC,H,1.011731,1.463751,0.000277,C,...,1.506668,0.356932,0.014849,1.552546,0.009594,1.469782,0.0017,1.783041,1.091946,1.783158


In [4]:
data['dist_x'] = abs(data['x_0']-data['x_1'])
data['dist_y'] = abs(data['y_0']-data['y_1'])
data['dist_z'] = abs(data['z_0']-data['z_1'])
data['dist'] = np.sqrt(data['dist_x']**2 + data['dist_y']**2 + data['dist_z']**2)

In [5]:
data['dist_x_molecule_mean'] = data.groupby('molecule_name')['dist_x'].transform('mean')
data['dist_x_molecule_std'] = data.groupby('molecule_name')['dist_x'].transform('std')
data['dist_x_molecule_min'] = data.groupby('molecule_name')['dist_x'].transform('min')
data['dist_x_molecule_max'] = data.groupby('molecule_name')['dist_x'].transform('max')
data['dist_x_molecule_median'] = data.groupby('molecule_name')['dist_x'].transform('median')
data['dist_x_molecule_skew'] = data.groupby('molecule_name')['dist_x'].transform('skew')

data['dist_y_molecule_mean'] = data.groupby('molecule_name')['dist_y'].transform('mean')
data['dist_y_molecule_std'] = data.groupby('molecule_name')['dist_y'].transform('std')
data['dist_y_molecule_min'] = data.groupby('molecule_name')['dist_y'].transform('min')
data['dist_y_molecule_max'] = data.groupby('molecule_name')['dist_y'].transform('max')
data['dist_y_molecule_median'] = data.groupby('molecule_name')['dist_y'].transform('median')
data['dist_y_molecule_skew'] = data.groupby('molecule_name')['dist_y'].transform('skew')

data['dist_z_molecule_mean'] = data.groupby('molecule_name')['dist_z'].transform('mean')
data['dist_z_molecule_std'] = data.groupby('molecule_name')['dist_z'].transform('std')
data['dist_z_molecule_min'] = data.groupby('molecule_name')['dist_z'].transform('min')
data['dist_z_molecule_max'] = data.groupby('molecule_name')['dist_z'].transform('max')
data['dist_z_molecule_median'] = data.groupby('molecule_name')['dist_z'].transform('median')
data['dist_z_molecule_skew'] = data.groupby('molecule_name')['dist_z'].transform('skew')

data['dist_molecule_mean'] = data.groupby('molecule_name')['dist'].transform('mean')
data['dist_molecule_std'] = data.groupby('molecule_name')['dist'].transform('std')
data['dist_molecule_min'] = data.groupby('molecule_name')['dist'].transform('min')
data['dist_molecule_max'] = data.groupby('molecule_name')['dist'].transform('max')
data['dist_molecule_median'] = data.groupby('molecule_name')['dist'].transform('median')
data['dist_molecule_skew'] = data.groupby('molecule_name')['dist'].transform('skew')

data['dist_x_molecule_type_mean'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('mean')
data['dist_x_molecule_type_std'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('std')
data['dist_x_molecule_type_min'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('min')
data['dist_x_molecule_type_max'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('max')
data['dist_x_molecule_type_median'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('median')
data['dist_x_molecule_type_skew'] = data.groupby(['molecule_name', 'type'])['dist_x'].transform('skew')

data['dist_y_molecule_type_mean'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('mean')
data['dist_y_molecule_type_std'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('std')
data['dist_y_molecule_type_min'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('min')
data['dist_y_molecule_type_max'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('max')
data['dist_y_molecule_type_median'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('median')
data['dist_y_molecule_type_skew'] = data.groupby(['molecule_name', 'type'])['dist_y'].transform('skew')

data['dist_z_molecule_type_mean'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('mean')
data['dist_z_molecule_type_std'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('std')
data['dist_z_molecule_type_min'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('min')
data['dist_z_molecule_type_max'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('max')
data['dist_z_molecule_type_median'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('median')
data['dist_z_molecule_type_skew'] = data.groupby(['molecule_name', 'type'])['dist_z'].transform('skew')

data['dist_molecule_type_mean'] = data.groupby(['molecule_name', 'type'])['dist'].transform('mean')
data['dist_molecule_type_std'] = data.groupby(['molecule_name', 'type'])['dist'].transform('std')
data['dist_molecule_type_min'] = data.groupby(['molecule_name', 'type'])['dist'].transform('min')
data['dist_molecule_type_max'] = data.groupby(['molecule_name', 'type'])['dist'].transform('max')
data['dist_molecule_type_median'] = data.groupby(['molecule_name', 'type'])['dist'].transform('median')
data['dist_molecule_type_skew'] = data.groupby(['molecule_name', 'type'])['dist'].transform('skew')

data.fillna(0.0, inplace=True)

In [6]:
atom_molecule = pd.crosstab(structure['molecule_name'], structure['atom'], normalize='index')
bond_molecule = pd.crosstab(data['molecule_name'], data['type'], normalize='index')

In [5]:
neighbour = pd.DataFrame({
    'molecule_name': np.hstack([data['molecule_name'], data['molecule_name']]),
    'atom_index_0': np.hstack([data['atom_index_0'], data['atom_index_1']]),
    'atom_index_1': np.hstack([data['atom_index_1'], data['atom_index_0']]),
    'atom_0': np.hstack([data['atom_0'], data['atom_1']]),
    'atom_1': np.hstack([data['atom_1'], data['atom_0']]),
    'type': np.hstack([data['type'], data['type']]),
    'dist_x': np.hstack([data['dist_x'], data['dist_x']]),
    'dist_y': np.hstack([data['dist_y'], data['dist_y']]),
    'dist_z': np.hstack([data['dist_z'], data['dist_z']]),
    'dist': np.hstack([data['dist'], data['dist']]),
}, columns=['molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'atom_1', 'type', 'dist_x', 'dist_y', 'dist_z', 'dist'])

In [10]:
bond_molecule_atom = pd.crosstab([neighbour['molecule_name'], neighbour['atom_index_0']], neighbour['type'], normalize='index')
atom_molecule_atom = pd.crosstab([neighbour['molecule_name'], neighbour['atom_index_0']], neighbour['atom_1'], normalize='index')

In [8]:
atom_property = pd.concat([
    neighbour.groupby(['molecule_name', 'atom_index_0'])['atom_index_1'].count().rename("neighbour_count_molecule_atom"),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].mean().rename('neighbour_mean_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].std().rename('neighbour_std_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].min().rename('neighbour_min_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].max().rename('neighbour_max_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].median().rename('neighbour_median_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist'].skew().rename('neighbour_skew_dist_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].mean().rename('neighbour_mean_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].std().rename('neighbour_std_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].min().rename('neighbour_min_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].max().rename('neighbour_max_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].median().rename('neighbour_median_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_x'].skew().rename('neighbour_skew_dist_x_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].mean().rename('neighbour_mean_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].std().rename('neighbour_std_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].min().rename('neighbour_min_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].max().rename('neighbour_max_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].median().rename('neighbour_median_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_y'].skew().rename('neighbour_skew_dist_y_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].mean().rename('neighbour_mean_dist_z_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].std().rename('neighbour_std_dist_z_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].min().rename('neighbour_min_dist_z_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].max().rename('neighbour_max_dist_z_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].median().rename('neighbour_median_dist_z_molecule_atom'),
    neighbour.groupby(['molecule_name', 'atom_index_0'])['dist_z'].skew().rename('neighbour_skew_dist_z_molecule_atom'),
], axis=1)
atom_property.fillna(0.0, inplace=True)

In [11]:
atom_property = pd.concat([atom_property, bond_molecule_atom.add_prefix("neighbour_"), atom_molecule_atom.add_prefix("neighbour_")], axis=1)

In [12]:
data = data.merge(atom_molecule, how='left', left_on='molecule_name', right_index=True)\
.merge(bond_molecule, how='left', left_on='molecule_name', right_index=True)\
.merge(atom_property.add_suffix("_0"), how='left', left_on=['molecule_name', 'atom_index_0'], right_index=True)\
.merge(atom_property.add_suffix("_1"), how='left', left_on=['molecule_name', 'atom_index_1'], right_index=True)

In [18]:
data.iloc[:4658147, :].drop(columns=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1']).to_csv("train_with_feature.csv", index=False)
data.iloc[4658147:, :].drop(columns=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1']).to_csv("test_with_feature.csv", index=False)

In [4]:
train_with_feature = pd.read_csv("train_with_feature.csv")
test_with_feature = pd.read_csv("test_with_feature.csv")
data = pd.concat([train_with_feature, test_with_feature], ignore_index=True)
cos_feature = pd.read_csv("cos_feature.csv")
data = pd.concat([cos_feature, data], axis=1)

NameError: name 'cos_feature' is not defined

In [6]:
cos_feature = pd.read_csv("cos_feature.csv")
data = pd.concat([cos_feature, data], axis=1)

In [8]:
cos_feature.columns

Index(['molecule_name', 'atom_index_0', 'atom_index_1', 'dist_0_n', 'dist_1_n',
       'dist_0_f', 'dist_1_f', 'dist_0_c', 'dist_1_c', 'cos_a0n0_a1n1',
       'cos_a0f0_a1f1', 'cos_a0c_a1c', 'cos_a0n0_a0a1', 'cos_a1n1_a0a1',
       'cos_a0f0_a0a1', 'cos_a1f1_a0a1'],
      dtype='object')

In [7]:
data.head()

,molecule_name,atom_index_0,atom_index_1,dist_0_n,dist_1_n,dist_0_f,dist_1_f,dist_0_c,dist_1_c,cos_a0n0_a1n1,...,neighbour_1JHN_1,neighbour_2JHC_1,neighbour_2JHH_1,neighbour_2JHN_1,neighbour_3JHC_1,neighbour_3JHH_1,neighbour_3JHN_1,neighbour_C_1,neighbour_H_1,neighbour_N_1
0,dsgdb9nsd_000001,1,0,1.091953,1.091946,1.783157,1.091953,1.091945,0.000012,0.333335,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,1.00,0.0
1,dsgdb9nsd_000001,1,2,1.091953,1.091952,1.783157,1.783158,1.091945,1.091946,-0.333287,...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.75,0.0
2,dsgdb9nsd_000001,1,3,1.091953,1.091946,1.783157,1.783158,1.091945,1.091954,-0.333335,...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.75,0.0
3,dsgdb9nsd_000001,1,4,1.091953,1.091948,1.783157,1.783157,1.091945,1.091954,-0.333347,...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.25,0.75,0.0
4,dsgdb9nsd_000001,2,0,1.091952,1.091946,1.783158,1.091953,1.091946,0.000012,0.333352,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,1.00,0.0


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(data[[
    'dist_0_n',
    'dist_1_n',
    #'dist_0_f',
    #'dist_1_f',
    #'dist_0_c',
    #'dist_1_c',
    'cos_a0n0_a1n1',
    'cos_a0f0_a1f1',
    'cos_a0c_a1c',
    'cos_a0n0_a0a1',
    'cos_a1n1_a0a1',
    'cos_a0f0_a0a1',
    'cos_a1f1_a0a1',
    'dist_x', 
    'dist_y', 
    'dist_z', 
    'dist',
    'dist_x_molecule_mean',
    'dist_x_molecule_std',
    'dist_x_molecule_min',
    'dist_x_molecule_max',
    'dist_x_molecule_median',
    'dist_x_molecule_skew',
    'dist_y_molecule_mean',
    'dist_y_molecule_std',
    'dist_y_molecule_min',
    'dist_y_molecule_max',
    'dist_y_molecule_median',
    'dist_y_molecule_skew',
    'dist_z_molecule_mean',
    'dist_z_molecule_std',
    'dist_z_molecule_min',
    'dist_z_molecule_max',
    'dist_z_molecule_median',
    'dist_z_molecule_skew',
    'dist_molecule_mean',
    'dist_molecule_std',
    'dist_molecule_min',
    'dist_molecule_max',
    'dist_molecule_median',
    'dist_molecule_skew',
    'dist_x_molecule_type_mean',
    'dist_x_molecule_type_std',
    'dist_x_molecule_type_min',
    'dist_x_molecule_type_max',
    'dist_x_molecule_type_median',
    'dist_x_molecule_type_skew',
    'dist_y_molecule_type_mean',
    'dist_y_molecule_type_std',
    'dist_y_molecule_type_min',
    'dist_y_molecule_type_max',
    'dist_y_molecule_type_median',
    'dist_y_molecule_type_skew',
    'dist_z_molecule_type_mean',
    'dist_z_molecule_type_std',
    'dist_z_molecule_type_min',
    'dist_z_molecule_type_max',
    'dist_z_molecule_type_median',
    'dist_z_molecule_type_skew',
    'dist_molecule_type_mean',
    'dist_molecule_type_std',
    'dist_molecule_type_min',
    'dist_molecule_type_max',
    'dist_molecule_type_median',
    'dist_molecule_type_skew',
    'neighbour_count_molecule_atom_0',
    'neighbour_mean_dist_molecule_atom_0',
    'neighbour_std_dist_molecule_atom_0',
    'neighbour_min_dist_molecule_atom_0',
    'neighbour_max_dist_molecule_atom_0',
    'neighbour_median_dist_molecule_atom_0',
    'neighbour_skew_dist_molecule_atom_0',
    'neighbour_mean_dist_x_molecule_atom_0',
    'neighbour_std_dist_x_molecule_atom_0',
    'neighbour_min_dist_x_molecule_atom_0',
    'neighbour_max_dist_x_molecule_atom_0',
    'neighbour_median_dist_x_molecule_atom_0',
    'neighbour_skew_dist_x_molecule_atom_0',
    'neighbour_mean_dist_y_molecule_atom_0',
    'neighbour_std_dist_y_molecule_atom_0',
    'neighbour_min_dist_y_molecule_atom_0',
    'neighbour_max_dist_y_molecule_atom_0',
    'neighbour_median_dist_y_molecule_atom_0',
    'neighbour_skew_dist_y_molecule_atom_0',
    'neighbour_mean_dist_z_molecule_atom_0',
    'neighbour_std_dist_z_molecule_atom_0',
    'neighbour_min_dist_z_molecule_atom_0',
    'neighbour_max_dist_z_molecule_atom_0',
    'neighbour_median_dist_z_molecule_atom_0',
    'neighbour_skew_dist_z_molecule_atom_0',
    'neighbour_count_molecule_atom_1',
    'neighbour_mean_dist_molecule_atom_1',
    'neighbour_std_dist_molecule_atom_1',
    'neighbour_min_dist_molecule_atom_1',
    'neighbour_max_dist_molecule_atom_1',
    'neighbour_median_dist_molecule_atom_1',
    'neighbour_skew_dist_molecule_atom_1',
    'neighbour_mean_dist_x_molecule_atom_1',
    'neighbour_std_dist_x_molecule_atom_1',
    'neighbour_min_dist_x_molecule_atom_1',
    'neighbour_max_dist_x_molecule_atom_1',
    'neighbour_median_dist_x_molecule_atom_1',
    'neighbour_skew_dist_x_molecule_atom_1',
    'neighbour_mean_dist_y_molecule_atom_1',
    'neighbour_std_dist_y_molecule_atom_1',
    'neighbour_min_dist_y_molecule_atom_1',
    'neighbour_max_dist_y_molecule_atom_1',
    'neighbour_median_dist_y_molecule_atom_1',
    'neighbour_skew_dist_y_molecule_atom_1',
    'neighbour_mean_dist_z_molecule_atom_1',
    'neighbour_std_dist_z_molecule_atom_1',
    'neighbour_min_dist_z_molecule_atom_1',
    'neighbour_max_dist_z_molecule_atom_1',
    'neighbour_median_dist_z_molecule_atom_1',
    'neighbour_skew_dist_z_molecule_atom_1',
]])

X = np.hstack([X, data[['C', 'F', 'H', 'N', 'O', '1JHC','1JHN','2JHC','2JHH','2JHN','3JHC','3JHH','3JHN',
                        'neighbour_C_0', 'neighbour_H_0', 'neighbour_N_0', 'neighbour_1JHC_0','neighbour_1JHN_0',
                        'neighbour_2JHC_0','neighbour_2JHH_0','neighbour_2JHN_0','neighbour_3JHC_0','neighbour_3JHH_0',
                        'neighbour_3JHN_0','neighbour_C_1', 'neighbour_H_1', 'neighbour_N_1', 'neighbour_1JHC_1','neighbour_1JHN_1',
                        'neighbour_2JHC_1','neighbour_2JHH_1','neighbour_2JHN_1','neighbour_3JHC_1','neighbour_3JHH_1',
                        'neighbour_3JHN_1']].values])

/home/ike/.miniconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ike/.miniconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [20]:
X[:1, :]

array([[ -1.42255436,  -0.14231097,  -1.42016288,  -1.72243022,
         -2.66140818,  -2.37933688,  -0.47894752,  -4.07950144,
         -4.92992023,  -2.89224422,  -0.53637718,  -5.93180051,
         -2.2634972 ,  -1.72472864,  -0.82096231,  -2.66751774,
         -8.77191017,  -9.92087517,   0.84239549, -17.90623758,
         -1.49858215,  -1.0166351 ,  -0.68574818,  -1.43237893,
         -1.30322565,  -1.14748866,   0.40633444,  -1.30994708,
         -1.62588108,  -0.59314269,  -0.70803739,  -1.52739137,
         -1.79084484,  -1.18201507,  -1.87650995,  -1.68993602,
          0.2       ,   0.        ,   0.8       ,   0.        ,
          0.        ]])

In [16]:
Xtest = X[4658147:, :]
X = X[:4658147, :]

In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

params = {'alpha': np.logspace(-2, 2, 10)}
folds = KFold(n_splits=5, random_state=49)
rr = Ridge()

y = train.scalar_coupling_constant

In [17]:
loss = []
model = {}

for bond in pd.unique(train['type']):
    Xb = X[train['type'] == bond]
    Yb = y[train['type'] == bond]
    grid_rr = GridSearchCV(rr, params, scoring='neg_mean_absolute_error', cv=folds, return_train_score=True, )
    grid_rr.fit(Xb, Yb)
    model[bond] = grid_rr
    loss.append(grid_rr.best_score_)

In [18]:
print(f"Final loss: {sum(map(lambda itm: np.log(-itm), loss))/8}")

Final loss: 0.5623235514339577


In [14]:
print(f"Final loss: {sum(map(lambda itm: np.log(-itm), loss))/8}")

Final loss: 0.5608561246517281


In [25]:
pd.DataFrame({
    'bond': pd.unique(train['type']),
    'loss': loss
}, columns=['bond', 'loss'])

,bond,loss
0,1JHC,-8.891959
1,2JHH,-1.953040
2,1JHN,-3.655107
3,2JHN,-2.740578
4,2JHC,-2.643697
5,3JHH,-2.999466
6,3JHC,-2.268999
7,3JHN,-0.921089


In [ ]:
submission = pd.read_csv(f"{DATA}sample_submission.csv")
for bond in pd.unique(train['type']):
    Xt = Xtest[test['type'] == bond]
    Yt = model[bond].predict(Xt)
    submission.loc[test['type'] == bond, 'scalar_coupling_constant'] = Yt

In [ ]:
submission.to_csv("submission_07_27_02.csv", index=False)